In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [43]:
df = pd.read_csv(r'C:\Users\absid\Downloads\v shale\Volvo T2 vsh - Copy.csv')
df.head()

,DEPTH,CALI,DRHO,DT,DTS,NPHI,PEF,ROP,RT,KLOGH,PHIF,VSH
0,4335.0,8.703,0.057,82.677,140.366,0.214,6.779,9.729,2.194,0.001,0.091,0.794
1,4335.1,8.687,0.057,81.456,139.508,0.216,6.800,10.004,2.261,0.001,0.088,0.785
2,4335.2,8.672,0.052,80.836,138.845,0.214,6.869,10.279,2.249,0.001,0.089,0.783
3,4335.3,8.672,0.046,81.743,138.684,0.209,6.956,10.554,2.231,0.001,0.090,0.765
4,4335.4,8.693,0.042,82.183,139.234,0.202,7.019,11.559,2.231,0.001,0.090,0.763


In [44]:
df.corr()

,DEPTH,CALI,DRHO,DT,DTS,NPHI,PEF,ROP,RT,KLOGH,PHIF,VSH
DEPTH,1.000000,0.032605,0.100498,-0.324598,-0.322210,-0.467721,-0.501766,-0.422978,0.074426,0.153039,0.229539,-0.528109
CALI,0.032605,1.000000,0.251472,0.016840,0.011059,0.124937,0.393339,0.067570,0.023137,-0.289878,-0.530702,0.317055
DRHO,0.100498,0.251472,1.000000,-0.060744,-0.033966,-0.046204,-0.053277,-0.238878,0.130843,-0.176374,-0.313360,0.113530
DT,-0.324598,0.016840,-0.060744,1.000000,0.881260,0.804261,0.067896,0.401733,-0.469102,-0.143374,0.276245,0.541467
DTS,-0.322210,0.011059,-0.033966,0.881260,1.000000,0.760345,0.075865,0.414588,-0.440304,-0.113304,0.254439,0.534102
NPHI,-0.467721,0.124937,-0.046204,0.804261,0.760345,1.000000,0.421080,0.526567,-0.390790,-0.195397,-0.007515,0.658549
PEF,-0.501766,0.393339,-0.053277,0.067896,0.075865,0.421080,1.000000,0.401694,-0.007826,-0.341239,-0.696362,0.431822
ROP,-0.422978,0.067570,-0.238878,0.401733,0.414588,0.526567,0.401694,1.000000,-0.329010,-0.128723,-0.006243,0.276571
RT,0.074426,0.023137,0.130843,-0.469102,-0.440304,-0.390790,-0.007826,-0.329010,1.000000,-0.044302,-0.277775,-0.133654
KLOGH,0.153039,-0.289878,-0.176374,-0.143374,-0.113304,-0.195397,-0.341239,-0.128723,-0.044302,1.000000,0.466457,-0.440315


In [45]:
# selecting features and label
X = df.iloc[:, :-1]  # features
y = df.iloc[:,-1]  # label
X.shape, y.shape

((1780, 11), (1780,))

In [46]:
# spliting into training and testing set for both X and y
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=27)


In [47]:
# function to convert seconds to into hours, minutes and seconds
# this function is used to measure the time taking by the models
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [57]:
# Loading the svr model from svm and GridSearchCV from model_selection
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
import time


regr = GridSearchCV(
    ExtraTreesRegressor(), 
    {
        'n_estimators' : [200,250,300,350,400],
        'max_features': ['sqrt','log2'],
        'criterion': ['squared_error','absolute_error',],
        'max_depth': [8,10,12,14],
        #'min_samples_split': [2,3,4,5],
        #'min_samples_leaf': [1,3,6,9],
        # 'min_weight_fraction_leaf': [0, 1,2,3],
        # 'max_leaf_nodes': [1,2,3,4,5],
    }, 
    cv=5, 
    scoring=['neg_mean_squared_error', 'r2'], 
    refit='r2',
    verbose=1)

start = time.time()
# enter your code below this line to calculate time it take

#Fit the model
regr.fit(X_train, y_train)

# enter you code above this line  to calculate time it take   
end = time.time()

print('run_time:', convert(end-start), 'h:m:s')


Fitting 5 folds for each of 80 candidates, totalling 400 fits
run_time: 0:36:42 h:m:s


In [58]:
# printing the best combination of parameter that perform the best
print(regr.best_params_)

# the best performing score
print(regr.best_score_)

{'criterion': 'squared_error', 'max_depth': 14, 'max_features': 'log2', 'n_estimators': 350}
0.9830217179483783



        'n_estimators' : [10, 50, 100, 150, 200, 300],
        'max_features': ['sqrt', 'log2', None],
        'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        'max_depth': [1,2,3,None],
        'min_samples_split': [2,3,4,5],
        'min_samples_leaf': [1,3,6,9],

In [59]:
import os

# Create the output/cnn directory if it doesn't exist
output_dir = "./output/extra_trees"
os.makedirs(output_dir, exist_ok=True)

In [60]:
# converting the results to a readable format using dataFrame
df = pd.DataFrame(regr.cv_results_)
# getting all parameter combinations and their performance result
result = df[['param_n_estimators', 'param_criterion', 'param_max_features', 'mean_test_r2', 'mean_test_neg_mean_squared_error']]
result.to_csv('./output/extra_trees/performance_result_for_extra_trees_gridsearchCV.csv')
result

,param_n_estimators,param_criterion,param_max_features,mean_test_r2,mean_test_neg_mean_squared_error
0,200,squared_error,sqrt,0.946169,-0.003519
1,250,squared_error,sqrt,0.946441,-0.003506
2,300,squared_error,sqrt,0.944420,-0.003634
3,350,squared_error,sqrt,0.945361,-0.003576
4,400,squared_error,sqrt,0.945213,-0.003584
...,...,...,...,...,...
75,200,absolute_error,log2,0.981713,-0.001188
76,250,absolute_error,log2,0.981547,-0.001199
77,300,absolute_error,log2,0.982369,-0.001144
78,350,absolute_error,log2,0.981948,-0.001168


In [61]:
import time
from sklearn.ensemble import ExtraTreesRegressor

regressor = ExtraTreesRegressor(n_estimators=regr.best_params_['n_estimators'], 
                                criterion=regr.best_params_['criterion'],
                                max_features=regr.best_params_['max_features'],
                                max_depth=regr.best_params_['max_depth'],
                            
                                verbose=1)

start = time.time() # starting of the time
regressor.fit(X_train, y_train) # fit/train the model

end = time.time() # ending of the time
print('run_time:', convert(end - start), 'h:m:s')


pred_1 = regressor.predict(X_train) # prediction for training set
pred_2 = regressor.predict(X_test) # prediction for testing set
pred_3 = regressor.predict(X) # prediction for whole dataset

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


run_time: 0:00:02 h:m:s


[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed:    0.2s finished


In [62]:
# saving training, testing and y data with their predicted data in a excel file with different sheets
a = np.stack([y_train, pred_1], axis=1) # training
a = pd.DataFrame(a, columns=['y_train', 'y_train_pred'])

b = np.stack([y_test, pred_2], axis=1) # testing 
b = pd.DataFrame(b, columns=['y_test', 'y_test_pred'])

c = np.stack([y, pred_3], axis=1) # whole data
c = pd.DataFrame(c, columns=['y', 'y_pred'])

with pd.ExcelWriter("./output/extra_trees/predicted_extra_trees_model.xlsx") as writer:
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    a.to_excel(writer, sheet_name="training", index=True)
    b.to_excel(writer, sheet_name="testing", index=True)
    c.to_excel(writer, sheet_name="whole_data", index=True)

In [63]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error

train_r2 = r2_score(y_train,pred_1) # r2_score for training set
train_mae = mean_absolute_error(y_train, pred_1) # mean absolute error for training set
train_mse = mean_squared_error(y_train, pred_1) # mean squared error for training set
train_mape = mean_absolute_percentage_error(y_train, pred_1) # mean_absolute_percentage_error for training set
train_ev = explained_variance_score(y_train, pred_1) 
train_maxE = max_error(y_train, pred_1)
train_minE = min(abs(y_train - pred_1))

test_r2 = r2_score(y_test, pred_2) # r2_score for testing set
test_mae = mean_absolute_error(y_test, pred_2) # mean absolute error for testing set
test_mse = mean_squared_error(y_test, pred_2) # mean squared error for testing set
test_mape = mean_absolute_percentage_error(y_test, pred_2) # mean_absolute_percentage_error for testing set
test_ev = explained_variance_score(y_test, pred_2) 
test_maxE = max_error(y_test, pred_2)
test_minE = min(abs(y_test - pred_2))

r2 = r2_score(y, pred_3) # r2_score for whole dataset
mae = mean_absolute_error(y, pred_3) # mean absolute error for whole dataset
mse = mean_squared_error(y, pred_3) # mean squared error for whole dataset
mape = mean_absolute_percentage_error(y, pred_3) # mean_absolute_percentage_error for whole dataset
ev = explained_variance_score(y, pred_3) 
maxE = max_error(y, pred_3)
minE = min(abs(y - pred_3))

metrics = {
'performance_metrics': ['R2', 'MAE', 'MSE', 'MAPE', 'EV', 'maxE', 'minE'],
'training': [train_r2, train_mae, train_mse, train_mape, train_ev, train_maxE, train_minE],
'testing': [test_r2, test_mae, test_mse, test_mape, test_ev, test_maxE, test_minE],
'whole': [r2, mae, mse, mape, ev, maxE, minE]
}

performance_metrics = pd.DataFrame(metrics)
performance = performance_metrics.transpose()
performance.to_csv('./output/extra_trees/performance_extra_trees.csv')
performance

,0,1,2,3,4,5,6
performance_metrics,R2,MAE,MSE,MAPE,EV,maxE,minE
training,0.997892,0.008475,0.000139,0.042479,0.997892,0.057947,0.000004
testing,0.981894,0.020467,0.001203,0.092077,0.981898,0.213891,0.00012
whole,0.99306,0.012072,0.000458,0.057359,0.99306,0.213891,0.000004


In [64]:
plt.figure(figsize=(9, 5))

sns.regplot(x=y_test, y=pred_2, scatter_kws={"color": "C0"}, line_kws={"color": "C1"})
plt.text(3775, 14389, "$R^2:$ {:.3f}".format(test_r2), fontsize=17)

plt.xlabel("Actual VSH", fontsize=20)
plt.ylabel("Predicted VSH", fontsize=20)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)

plt.tight_layout()
#plt.savefig("./output/extra_trees/regression_plot_extra_trees.png", dpi=300)
plt.show()

C:\Users\absid\AppData\Local\Temp\ipykernel_852\2395663934.py:11: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


ValueError: Image size of 2880720x6230009 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 900x500 with 1 Axes>

In [65]:
#from sklearn.externals import joblib
from joblib import dump, load
dump(regressor, './output/extra_trees/trained_extra_trees_model.joblib')

#clf = load('trained_linear_regression_model.joblib')

['./output/extra_trees/trained_extra_trees_model.joblib']